In [1]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
import time
import xarray as xr
from netCDF4 import Dataset

import keras
from keras.layers import Input, Dense, merge, Embedding, Flatten, Concatenate,Conv2D,BatchNormalization,Dropout,MaxPooling2D
from keras.models import Model, Sequential
from keras.optimizers import Adam, SGD
import keras.backend as K
from keras.callbacks import EarlyStopping

from cartopy import config
import cartopy.crs as ccrs
import cartopy.feature as cf
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import palettable

import random 
random.seed(1) #for reproduceability. 
import matplotlib.ticker as mticker
from sklearn import datasets, linear_model, metrics 
# import utils

if keras.backend.backend() == 'tensorflow':
    from tensorflow import erf
else:
    from theano.tensor import erf
# import utils

Using TensorFlow backend.
/Users/will/anaconda2/envs/post_process/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/will/anaconda2/envs/post_process/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/will/anaconda2/envs/post_process/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/User

In [302]:
def crps_cost_function(y_true, y_pred, theano=False):
    """Compute the CRPS cost function for a normal distribution defined by
    the mean and standard deviation.
    Code inspired by Kai Polsterer (HITS).
    Args:
        y_true: True values
        y_pred: Tensor containing predictions: [mean, std]
        theano: Set to true if using this with pure theano.
    Returns:
        mean_crps: Scalar with mean CRPS over batch
    """

    # Split input
    mu = y_pred[:, 0]
    sigma = y_pred[:, 1]
    # Ugly workaround for different tensor allocation in keras and theano
    if not theano:
        y_true = y_true[:, 0]   # Need to also get rid of axis 1 to match!

    # To stop sigma from becoming negative we first have to 
    # convert it the the variance and then take the square
    # root again. 
    var = K.square(sigma)
    # The following three variables are just for convenience
    loc = (y_true - mu) / K.sqrt(var)
    phi = 1.0 / np.sqrt(2.0 * np.pi) * K.exp(-K.square(loc) / 2.0)
    Phi = 0.5 * (1.0 + erf(loc / np.sqrt(2.0)))
    # First we will compute the crps for each input/target pair
    crps =  K.sqrt(var) * (loc * (2. * Phi - 1.) + 2 * phi - 1. / np.sqrt(np.pi))
    # Then we take the mean. The cost is now a scalar
    return K.mean(crps)



def build_EMOS_network_keras(compile=False, optimizer='sgd', lr=0.1):
    """Build (and maybe compile) EMOS network in keras.
    Args:
        compile: If true, compile model
        optimizer: String of keras optimizer
        lr: learning rate
    Returns:
        model: Keras model
    """
    mean_in = Input(shape=(1,))
    std_in = Input(shape=(1,))
    mean_out = Dense(1, activation='linear')(mean_in)
    std_out = Dense(1, activation='linear')(std_in)
    x = keras.layers.concatenate([mean_out, std_out], axis=1)
    model = Model(inputs=[mean_in, std_in], outputs=x)

    if compile:
        opt = keras.optimizers.__dict__[optimizer](lr=lr)
        model.compile(optimizer=opt, loss=crps_cost_function)
    return model



def build_NN_network_keras(compile=False, optimizer='sgd', lr=0.1):
    """Build (and maybe compile) EMOS network in keras.
    Args:
        compile: If true, compile model
        optimizer: String of keras optimizer
        lr: learning rate
    Returns:
        model: Keras model
    """
    mean_in = Input(shape=(1,))
    std_in = Input(shape=(1,))
    mean_out = Dense(1, activation='relu')(mean_in)
    std_out = Dense(1, activation='relu')(std_in)
    x = keras.layers.concatenate([mean_out, std_out], axis=1)
    
    D1 = Dense(10, activation='relu')(x)
    D2 = Dense(2, activation='linear')(D1)
    
    model = Model(inputs=[mean_in, std_in], outputs=D2)

    if compile:
        opt = keras.optimizers.__dict__[optimizer](lr=lr)
        model.compile(optimizer=opt, loss=crps_cost_function)
    return model

In [331]:
AllDat = xr.open_zarr('/Users/will/Desktop/Haupt/PNA/CMAzar/')
AllDat=AllDat.to_dataframe()

In [332]:
AllDat

,Analysis,Center,Ense,Forecast_Date,Forecast_Lead,PNA
index,,,,,,
0,0.148891,CMA,0,2007051512,0,0.148173
1,0.634738,CMA,0,2007051512,1,0.592949
2,0.300384,CMA,0,2007051512,2,0.291869
3,0.246399,CMA,0,2007051512,3,0.204389
4,0.420670,CMA,0,2007051512,4,0.381550
...,...,...,...,...,...,...
235,0.000000,CMA,14,2020013112,11,-1.853610
236,0.000000,CMA,14,2020013112,12,-1.542850
237,0.000000,CMA,14,2020013112,13,-1.526760


In [347]:
#
leadtime=7 #days
monthz = 2 #month to post-process with EMOS 
AllDatLead = AllDat.loc[AllDat['Forecast_Lead']==leadtime]
#create a day,month,year column:

datetime_str = '09/19/18 13:55:26'
datetime_object = datetime.strptime(datetime_str, '%m/%d/%y %H:%M:%S')

dayz = []
mons = []
yrs = []

for bb,dat in enumerate((AllDatLead['Forecast_Date'])):
    datetime_object = datetime.strptime(str(dat), '%Y%m%d%H')
    yrs.append(datetime_object.year)
    mons.append(datetime_object.month)
    dayz.append(datetime_object.day)
    
AllDatLead['day']=dayz
AllDatLead['month']=mons
AllDatLead['year']=yrs

# #select the month we'd like to post-process
# AllDatMon = AllDatLead.loc[AllDatLead['month'] == monthz]
AllDatMon = AllDatLead


PNA_mean=[]
PNA_std=[]
PNA_obs=[]
For_Date=[]

#create mean and standard deviation for the Ensemble forecast: 
for nn,dat in enumerate(np.unique(AllDatMon['Forecast_Date'])):
    tt = AllDatMon[AllDatMon.Forecast_Date==dat].PNA
    PNA_mean.append(np.mean(AllDatMon[AllDatMon.Forecast_Date==dat].PNA))
    PNA_std.append(np.std(AllDatMon[AllDatMon.Forecast_Date==dat].PNA))
    PNA_obs.append(np.mean(AllDatMon[AllDatMon.Forecast_Date==dat].Analysis))
    For_Date.append(dat)
    
    break

/Users/will/anaconda2/envs/post_process/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/will/anaconda2/envs/post_process/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/will/anaconda2/envs/post_process/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [348]:
#create mean and standard deviation for the Ensemble forecast: 
PNA_mean=[]
PNA_std=[]
PNA_obs=[]
For_Date=[]
yz=[]
mz=[]
dz=[]

for nn,dat in enumerate(np.unique(AllDatMon['Forecast_Date'])):
    tt = AllDatMon[AllDatMon.Forecast_Date==dat].PNA
    PNA_mean.append(np.mean(AllDatMon[AllDatMon.Forecast_Date==dat].PNA))
    PNA_std.append(np.std(AllDatMon[AllDatMon.Forecast_Date==dat].PNA))
    PNA_obs.append(np.mean(AllDatMon[AllDatMon.Forecast_Date==dat].Analysis))
    dz.append(np.mean(AllDatMon[AllDatMon.Forecast_Date==dat].day))
    mz.append(np.mean(AllDatMon[AllDatMon.Forecast_Date==dat].month))
    yz.append(np.mean(AllDatMon[AllDatMon.Forecast_Date==dat].year))
    For_Date.append(dat)

    
d = {'PNA_mean': PNA_mean, 'PNA_std': PNA_std,'PNA_obs':PNA_obs,'Fore_Date':For_Date,
     'day':dz,'month':mz,'year':yz}
PNAdf  = pd.DataFrame(data=d)

In [349]:
PNA_train = PNAdf[PNAdf.year<=2017]
PNA_validate = PNAdf[(PNAdf.year>=2018)&(PNAdf.year<2019)]
PNA_test = PNAdf[(PNAdf.year>=2019)]

## Split: Train,Test,Validate

In [350]:
xm=np.array(PNA_train['PNA_mean'])
xs=np.array(PNA_train['PNA_std'])
y=np.array(PNA_train['PNA_obs'])

xm_v=np.array(PNA_validate['PNA_mean'])
xs_v=np.array(PNA_validate['PNA_std'])
y_v=np.array(PNA_validate['PNA_obs'])

xm_t=np.array(PNA_test['PNA_mean'])
xs_t=np.array(PNA_test['PNA_std'])
y_t=np.array(PNA_test['PNA_obs'])

In [351]:
emos = build_EMOS_network_keras(compile=True, optimizer='sgd', lr=0.1)
emos.summary()

Model: "model_31"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_61 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_62 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
dense_95 (Dense)                (None, 1)            2           input_61[0][0]                   
__________________________________________________________________________________________________
dense_96 (Dense)                (None, 1)            2           input_62[0][0]                   
___________________________________________________________________________________________

In [352]:
bn=20
epcs=100
#### KERAS CALLBACKS TO ADD to Training######
filp = '/where/your/best/model/is/saved'
svbst = keras.callbacks.callbacks.ModelCheckpoint(filp, monitor='val_loss', 
                                                  verbose=1, save_best_only=True, save_weights_only=False)
#add this to the callbacks in fit function to save the best model on your personal machine. 

earlystop = keras.callbacks.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, 
                                                    verbose=1, mode='auto', restore_best_weights=True) 
rdclr = keras.callbacks.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, 
                                                    mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

#### Fitting the Model ######
emos.fit([xm,xs],y,batch_size=bn,validation_data=[[xm_v,xs_v],y_v],epochs=epcs,callbacks=[earlystop,rdclr])

Train on 2993 samples, validate on 342 samples
Epoch 1/100
2993/2993 [==============================] - 1s 315us/step - loss: 0.2460 - val_loss: 0.2334
Epoch 2/100
2993/2993 [==============================] - 0s 49us/step - loss: 0.2435 - val_loss: 0.2330
Epoch 3/100
2993/2993 [==============================] - 0s 52us/step - loss: 0.2433 - val_loss: 0.2299
Epoch 4/100
2993/2993 [==============================] - 0s 49us/step - loss: 0.2433 - val_loss: 0.2311
Epoch 5/100
2993/2993 [==============================] - 0s 49us/step - loss: 0.2427 - val_loss: 0.2316
Epoch 6/100
2993/2993 [==============================] - 0s 49us/step - loss: 0.2424 - val_loss: 0.2288
Epoch 7/100
2993/2993 [==============================] - 0s 49us/step - loss: 0.2426 - val_loss: 0.2315
Epoch 8/100
2993/2993 [==============================] - 0s 50us/step - loss: 0.2420 - val_loss: 0.2286
Epoch 9/100
2993/2993 [==============================] - 0s 50us/step - loss: 0.2422 - val_loss: 0.2309
Epoch 10/100
299

## Evaluate

In [353]:
#make a prediction:
preds = emos.predict([xm_t, xs_t])

In [354]:
d = {'Forecast_Time': PNA_test.Fore_Date,'Obs': PNA_test.PNA_obs,'Emos_mean': preds[:,0],'Emos_std': preds[:,1],
    'Model_mean':PNA_test.PNA_mean,'Model_std':PNA_test.PNA_std}
results_df = pd.DataFrame(d)
#Sorting DataFrame by time and Station ID
results_df

,Forecast_Time,Obs,Emos_mean,Emos_std,Model_mean,Model_std
3335,2019010112,-0.102586,-0.102080,0.340791,-0.257838,0.236883
3336,2019010212,0.308736,0.151038,0.425267,0.052651,0.328747
3337,2019010312,0.198227,-0.161174,0.344373,-0.330326,0.240778
3338,2019010412,0.830801,0.490254,0.299558,0.468752,0.192043
3339,2019010512,0.519359,0.544322,0.306831,0.535075,0.199952
...,...,...,...,...,...,...
3696,2020012712,-0.039734,-0.153388,0.386627,-0.320775,0.286727
3697,2020012812,-0.020386,0.271076,0.381503,0.199896,0.281155
3698,2020012912,-0.035568,0.113824,0.360397,0.007002,0.258203
3699,2020013012,-0.036261,0.027995,0.436108,-0.098281,0.340536


In [355]:
crps_preds = emos.evaluate([xm_t,xs_t],y_t)
#jump through hoops to get data in the right form for loss function:
MODpna_pred = np.transpose(np.array([results_df.Model_mean,results_df.Model_std]))
crps_mod= keras.backend.eval(crps_cost_function(np.expand_dims(y_t,axis=1),MODpna_pred))

366/366 [==============================] - 0s 23us/step


In [356]:
print('Post-Processed with EMOS = CRPS:',crps_preds)
print('Raw Ensemble a Global = CRPS:',crps_mod)

Post-Processed with EMOS = CRPS: 0.21587287402543864
Raw Ensemble a Global = CRPS: 0.23820415125847338


## Try with a Neural Network

In [357]:
nn = build_NN_network_keras(compile=True, optimizer='sgd', lr=0.1)
nn.summary()

Model: "model_32"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_63 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_64 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
dense_97 (Dense)                (None, 1)            2           input_63[0][0]                   
__________________________________________________________________________________________________
dense_98 (Dense)                (None, 1)            2           input_64[0][0]                   
___________________________________________________________________________________________

In [358]:
bn=20
epcs=100
#### KERAS CALLBACKS TO ADD to Training######
filp = '/where/your/best/model/is/saved'
svbst = keras.callbacks.callbacks.ModelCheckpoint(filp, monitor='val_loss', 
                                                  verbose=1, save_best_only=True, save_weights_only=False)
#add this to the callbacks in fit function to save the best model on your personal machine. 

earlystop = keras.callbacks.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=20, 
                                                    verbose=1, mode='auto', restore_best_weights=True) 
rdclr = keras.callbacks.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, 
                                                    mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

#### Fitting the Model ######
nn.fit([xm,xs],y,batch_size=bn,validation_data=[[xm_v,xs_v],y_v],epochs=epcs,callbacks=[earlystop,rdclr])

Train on 2993 samples, validate on 342 samples
Epoch 1/100
2993/2993 [==============================] - 1s 349us/step - loss: 0.2714 - val_loss: 0.2620
Epoch 2/100
2993/2993 [==============================] - 0s 55us/step - loss: 0.2491 - val_loss: 0.2490
Epoch 3/100
2993/2993 [==============================] - 0s 56us/step - loss: 0.2445 - val_loss: 0.2521
Epoch 4/100
2993/2993 [==============================] - 0s 54us/step - loss: 0.2430 - val_loss: 0.2408
Epoch 5/100
2993/2993 [==============================] - 0s 56us/step - loss: 0.2433 - val_loss: 0.2378
Epoch 6/100
2993/2993 [==============================] - 0s 54us/step - loss: 0.2427 - val_loss: 0.2385
Epoch 7/100
2993/2993 [==============================] - 0s 54us/step - loss: 0.2426 - val_loss: 0.2365
Epoch 8/100
2993/2993 [==============================] - 0s 53us/step - loss: 0.2419 - val_loss: 0.2386
Epoch 9/100
2993/2993 [==============================] - 0s 53us/step - loss: 0.2427 - val_loss: 0.2376
Epoch 10/100
299

In [359]:
#make a prediction:
preds = nn.predict([xm_t, xs_t])

d = {'Forecast_Time': PNA_test.Fore_Date,'Obs': PNA_test.PNA_obs,'Emos_mean': preds[:,0],'Emos_std': preds[:,1],
    'Model_mean':PNA_test.PNA_mean,'Model_std':PNA_test.PNA_std}
results_df = pd.DataFrame(d)
#Sorting DataFrame by time and Station ID
results_df

,Forecast_Time,Obs,Emos_mean,Emos_std,Model_mean,Model_std
3335,2019010112,-0.102586,-0.122834,-0.373830,-0.257838,0.236883
3336,2019010212,0.308736,0.135451,-0.419044,0.052651,0.328747
3337,2019010312,0.198227,-0.179630,-0.367424,-0.330326,0.240778
3338,2019010412,0.830801,0.445675,-0.437097,0.468752,0.192043
3339,2019010512,0.519359,0.499227,-0.444837,0.535075,0.199952
...,...,...,...,...,...,...
3696,2020012712,-0.039734,-0.165725,-0.375875,-0.320775,0.286727
3697,2020012812,-0.020386,0.245338,-0.425561,0.199896,0.281155
3698,2020012912,-0.035568,0.089593,-0.403058,0.007002,0.258203
3699,2020013012,-0.036261,0.017699,-0.406307,-0.098281,0.340536


In [360]:
crps_preds = nn.evaluate([xm_t,xs_t],y_t)
#jump through hoops to get data in the right form for loss function:
MODpna_pred = np.transpose(np.array([results_df.Model_mean,results_df.Model_std]))
crps_mod= keras.backend.eval(crps_cost_function(np.expand_dims(y_t,axis=1),MODpna_pred))

366/366 [==============================] - 0s 25us/step


In [361]:
print('Post-Processed with EMOS = CRPS:',crps_preds)
print('Raw Ensemble a Global = CRPS:',crps_mod)

Post-Processed with EMOS = CRPS: 0.22985542292803363
Raw Ensemble a Global = CRPS: 0.23820415125847338
